<a href="https://colab.research.google.com/github/AndersonZangelmi/Exercicio_Python/blob/master/BD_API_GOV_AZ_OWNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run below commands in google colab
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.0
!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
# unzip it
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
# install findspark
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

In [5]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [6]:
import pyspark
print(pyspark.__version__)

3.0.1


In [7]:
!curl -X GET --header 'Accept: application/json' --header 'chave-api-dados: a656fa09df76dfd3de97a34a006c19f3' 'http://www.portaltransparencia.gov.br/api-de-dados/acordos-leniencia?pagina=1' > consulta_portal_transparencia.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5253    0  5253    0     0   4178      0 --:--:--  0:00:01 --:--:--  4178


In [8]:
#Instancia o contexto do Spark
sc = spark.sparkContext
#Realiza a leitura da nossa consulta realizada na API do Governo Federal sobre as Viagens
path = "/content/consulta_portal_transparencia.json"
#Declara dataframe df com o conteudo da api que estava no JSON
df = spark.read.json(path)

In [44]:
#Imprime a estrutura de dados do nosso dataframe
df.printSchema()
#Imprime uma amosta de dados ( para especificar as linhas basta colocar a quantidade entre ())
df.show()

root
 |-- cnpj: string (nullable = true)
 |-- dataFimAcordo: string (nullable = true)
 |-- dataInicioAcordo: string (nullable = true)
 |-- id: long (nullable = true)
 |-- nomeEmpresa: string (nullable = true)
 |-- nomeFantasia: string (nullable = true)
 |-- orgaoResponsavel: string (nullable = true)
 |-- quantidade: long (nullable = true)
 |-- razaoSocial: string (nullable = true)
 |-- situacaoAcordo: string (nullable = true)
 |-- ufEmpresa: string (nullable = true)

+------------------+-------------+----------------+-------+--------------------+--------------------+--------------------+----------+--------------------+--------------+---------+
|              cnpj|dataFimAcordo|dataInicioAcordo|     id|         nomeEmpresa|        nomeFantasia|    orgaoResponsavel|quantidade|         razaoSocial|situacaoAcordo|ufEmpresa|
+------------------+-------------+----------------+-------+--------------------+--------------------+--------------------+----------+--------------------+--------------

In [152]:
tabela = df.selectExpr("cnpj","dataFimAcordo","situacaoAcordo","nomeEmpresa")
tabela.show()

+------------------+-------------+--------------+--------------------+
|              cnpj|dataFimAcordo|situacaoAcordo|         nomeEmpresa|
+------------------+-------------+--------------+--------------------+
|14.811.848/0001-05|   31/12/2047|   Em Execução|OAS S.A. - EM REC...|
|42.150.391/0001-70|   30/01/2025|   Em Execução|         BRASKEM S.A|
|17.262.213/0001-94|   30/06/2031|   Em Execução|ANDRADE GUTIERREZ...|
|61.522.512/0001-02|   31/01/2038|   Em Execução|CONSTRUCOES E COM...|
|44.023.661/0001-08|   10/07/2038|   Em Execução|U T C ENGENHARIA ...|
|61.067.377/0001-52|   13/04/2020|   Em Execução|MULLEN LOWE BRASI...|
|68.915.891/0001-40|   25/06/2021|   Em Execução|TECHNIP BRASIL - ...|
|05.144.757/0001-72|   31/10/2039|   Em Execução|ODEBRECHT S.A. EM...|
|            EXSBM |   29/06/2030|   Em Execução|   SBM OFFSHORE N.V.|
|02.357.415/0001-42|   31/01/2046|   Em Execução|NOVA PARTICIPACOE...|
|      EXBILFINGER |   31/12/2019|   Em Execução|BILFINGER MASCHIN...|
|75.60

In [157]:
#Declara tabela temporaria no SparkSQL
tabela.createOrReplaceTempView("tabela")

In [190]:
import datetime as date
Fim_acordo = spark.sql("select cnpj , dataFimAcordo,  nomeEmpresa as Empresa from tabela order by nomeEmpresa asc")
Fim_acordo.show()

+------------------+-------------+--------------------+
|              cnpj|dataFimAcordo|             Empresa|
+------------------+-------------+--------------------+
|22.641.641/0001-68|   16/10/2019|AMBIENTAL ENGENHA...|
|17.262.213/0001-94|   30/06/2031|ANDRADE GUTIERREZ...|
|20.729.334/0001-08|   18/07/2021|ANDRI SOLUCOES CO...|
|      EXBILFINGER |   31/12/2019|BILFINGER MASCHIN...|
|42.150.391/0001-70|   30/01/2025|         BRASKEM S.A|
|61.522.512/0001-02|   31/01/2038|CONSTRUCOES E COM...|
|61.067.377/0001-52|   13/04/2020|MULLEN LOWE BRASI...|
|02.357.415/0001-42|   31/01/2046|NOVA PARTICIPACOE...|
|14.811.848/0001-05|   31/12/2047|OAS S.A. - EM REC...|
|05.144.757/0001-72|   31/10/2039|ODEBRECHT S.A. EM...|
|75.609.123/0001-23|   03/08/2020|OURO VERDE LOCACA...|
|            EXSBM |   29/06/2030|   SBM OFFSHORE N.V.|
|68.915.891/0001-40|   25/06/2021|TECHNIP BRASIL - ...|
|44.023.661/0001-08|   10/07/2038|U T C ENGENHARIA ...|
+------------------+-------------+--------------